# Random Walk in 1D: Standard Deviation vs Number of Steps

This notebook explores how the standard deviation of the final position changes with the number of steps in a 1D random walk.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats

## Single Random Walk

First, let's visualize a single random walk trajectory.

In [ ]:
def random_walk_1d(n_steps):
    """Generate a 1D random walk with n_steps."""
    steps = np.random.choice([-1, 1], size=n_steps)
    position = np.cumsum(steps)
    return np.concatenate([[0], position])

# Example walk
n_steps = 1000
walk = random_walk_1d(n_steps)

plt.figure(figsize=(12, 5))
plt.plot(walk)
plt.xlabel('Step')
plt.ylabel('Position')
plt.title(f'Single Random Walk ({n_steps} steps)')
plt.grid(True, alpha=0.3)
plt.show()

## Standard Deviation vs Number of Steps

Theory predicts that the standard deviation of the final position should scale as $\sigma \sim \sqrt{N}$, where $N$ is the number of steps.

In [ ]:
def measure_stddev(n_steps, n_walks=1000):
    """Measure standard deviation of final position for multiple walks."""
    final_positions = np.zeros(n_walks)
    for i in range(n_walks):
        walk = random_walk_1d(n_steps)
        final_positions[i] = walk[-1]
    return np.std(final_positions)

# Test different numbers of steps
step_counts = np.logspace(1, 4, 20, dtype=int)  # From 10 to 10,000 steps
stddevs = []
n_walks = 1000

print(f"Running {n_walks} walks for each step count...")
for n_steps in step_counts:
    stddev = measure_stddev(n_steps, n_walks)
    stddevs.append(stddev)
    print(f"N={n_steps:5d}, σ={stddev:.2f}")

stddevs = np.array(stddevs)

## Visualization and Analysis

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))

# Linear scale
ax1.scatter(step_counts, stddevs, label='Measured', alpha=0.6)
ax1.plot(step_counts, np.sqrt(step_counts), 'r--', label='Theoretical $\\sqrt{N}$', linewidth=2)
ax1.set_xlabel('Number of Steps (N)')
ax1.set_ylabel('Standard Deviation (σ)')
ax1.set_title('Standard Deviation vs Number of Steps')
ax1.legend()
ax1.grid(True, alpha=0.3)

# Log-log scale
ax2.loglog(step_counts, stddevs, 'o', label='Measured', alpha=0.6)
ax2.loglog(step_counts, np.sqrt(step_counts), 'r--', label='Theoretical $\\sqrt{N}$', linewidth=2)
ax2.set_xlabel('Number of Steps (N)')
ax2.set_ylabel('Standard Deviation (σ)')
ax2.set_title('Log-Log Plot')
ax2.legend()
ax2.grid(True, alpha=0.3, which='both')

plt.tight_layout()
plt.show()

## Power Law Fitting

Let's fit the data to verify the $\sqrt{N}$ relationship.

In [ ]:
# Fit power law: σ = a * N^b
# In log space: log(σ) = log(a) + b * log(N)
log_steps = np.log(step_counts)
log_stddevs = np.log(stddevs)

slope, intercept, r_value, p_value, std_err = stats.linregress(log_steps, log_stddevs)

print(f"\nPower Law Fit: σ = {np.exp(intercept):.3f} * N^{slope:.3f}")
print(f"Theoretical exponent: 0.5")
print(f"Measured exponent: {slope:.3f}")
print(f"R² = {r_value**2:.6f}")
print(f"Relative error: {abs(slope - 0.5) / 0.5 * 100:.2f}%")

## Distribution of Final Positions

Let's examine the distribution of final positions for a fixed number of steps.

In [ ]:
n_steps = 1000
n_walks = 5000
final_positions = np.array([random_walk_1d(n_steps)[-1] for _ in range(n_walks)])

plt.figure(figsize=(12, 5))
plt.hist(final_positions, bins=50, density=True, alpha=0.7, label='Measured')

# Overlay theoretical normal distribution
x = np.linspace(final_positions.min(), final_positions.max(), 100)
theoretical_std = np.sqrt(n_steps)
plt.plot(x, stats.norm.pdf(x, 0, theoretical_std), 'r-', linewidth=2, 
         label=f'Normal(0, $\\sqrt{{{n_steps}}}$)')

plt.xlabel('Final Position')
plt.ylabel('Probability Density')
plt.title(f'Distribution of Final Positions ({n_steps} steps, {n_walks} walks)')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

print(f"\nMeasured mean: {np.mean(final_positions):.2f}")
print(f"Measured std: {np.std(final_positions):.2f}")
print(f"Theoretical std: {theoretical_std:.2f}")